In [2]:
import pyodbc
import pyodbc
import pandas as pd
import os 
# 连接到SQL Server数据库
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=192.168.0.141;'
                      'DATABASE=MemberCenter;'
                      'UID=sa;'
                      'PWD=rootroot;')
# SQL 查询
sql = """
SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME not LIKE '%temp%'
"""
# 将查询结果读取到DataFrame中
df = pd.read_sql_query(sql, conn)
# 关闭连接
conn.close()
table_names = df['TABLE_NAME'].tolist()
# 连接到SQL Server数据库
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=192.168.0.141;'
                      'DATABASE=MemberCenter;'
                      'UID=sa;'
                      'PWD=rootroot;')
# 创建一个游标对象
cursor = conn.cursor()
# 获取数据库中所有的表名
table_names = []
# 读取member_data文件夹中的所有文件名
exist_tables = []
file_names = os.listdir('member_data')
# 遍历文件名列表
for file_name in file_names:
    # 获取文件名中的表名
    exist_table = file_name.split('.')[0]
    # 将表名添加到列表中
    exist_tables.append(exist_table)
for row in cursor.tables():
    if row.table_type == 'TABLE':
        table_names.append(row.table_name)
# 除去table_names中在exist_tables中已经存在的表名
table_names = list(set(table_names) - set(exist_tables))
# 初始化超时表和跑崩表的列表
timeout_tables = []
failed_tables = []
len(table_names)

C:\Users\xieyu\AppData\Local\Temp\ipykernel_27152\2374144928.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


29

In [ ]:
# 遍历所有的表
for table_name in table_names:
    print('我开始爬取表：', table_name)
    # SQL 查询
    sql = f"""
            DECLARE @TotalCount INT;
            DECLARE @DynamicSQL NVARCHAR(MAX);
            DECLARE @ColumnNames NVARCHAR(MAX);
            DECLARE @ColumnNamesConverted NVARCHAR(MAX);
            -- 获取总行数
            SELECT @TotalCount = COUNT(*) FROM {table_name};

            -- 获取字段名称列表和转换后的字段名称列表
            SELECT @ColumnNames = STUFF(
                (
                    SELECT ', ' + QUOTENAME(COLUMN_NAME)
                    FROM INFORMATION_SCHEMA.COLUMNS
                    WHERE TABLE_NAME = '{table_name}'
                    FOR XML PATH('')
                ), 1, 2, '');

            SELECT @ColumnNamesConverted = STUFF(
                (
                    SELECT ', CONVERT(NVARCHAR(MAX), ' + QUOTENAME(COLUMN_NAME) + ') AS ' + QUOTENAME(COLUMN_NAME)
                    FROM INFORMATION_SCHEMA.COLUMNS
                    WHERE TABLE_NAME = '{table_name}'
                    FOR XML PATH('')
                ), 1, 2, '');

            -- 动态构造查询语句
            SET @DynamicSQL = N'
                WITH ValueCounts AS (
                    SELECT 
                        COLUMN_NAME,
                        Value,
                        COUNT(*) AS ValueCount
                    FROM 
                        (SELECT ' + @ColumnNamesConverted + ' FROM {table_name}) AS ConvertedData
                        UNPIVOT
                        (
                            Value FOR COLUMN_NAME IN
                            (' + @ColumnNames + ')
                        ) AS UnpivotedData
                    WHERE
                        Value IS NOT NULL
                    GROUP BY
                        COLUMN_NAME, Value
                ), GroupedCounts AS (
                    SELECT 
                        COLUMN_NAME,
                        SUM(ValueCount) AS NonNullCount,
                        COUNT(Value) AS DistinctValueCount,
                        STUFF((
                            SELECT '', '' +  Value
                            FROM ValueCounts AS InnerValueCounts
                            WHERE InnerValueCounts.COLUMN_NAME = OuterValueCounts.COLUMN_NAME
                            FOR XML PATH('''')
                        ), 1, 2, '''') AS DistinctValues
                    FROM
                        ValueCounts AS OuterValueCounts
                    GROUP BY
                        COLUMN_NAME
                ), ColumnTypes AS (
                    SELECT 
                        COLUMN_NAME,
                        DATA_TYPE AS FieldType
                    FROM 
                        INFORMATION_SCHEMA.COLUMNS
                    WHERE 
                        TABLE_NAME = ''{table_name}''
                )
                SELECT 
                    ''{table_name}'' AS TableName,
                    gc.COLUMN_NAME AS FieldName,
                    ct.FieldType,
                    gc.NonNullCount * 100.0 / ' + CAST(@TotalCount AS NVARCHAR) + ' AS NonNullPercentage,
                    gc.DistinctValueCount,
                    CASE 
                        WHEN gc.DistinctValueCount <= 120 THEN gc.DistinctValues
                        ELSE ''大于120种类''
                    END AS DistinctValues
                FROM 
                    GroupedCounts AS gc
                    JOIN ColumnTypes AS ct ON gc.COLUMN_NAME = ct.COLUMN_NAME;
            ';

            -- 执行动态查询语句
            EXEC sp_executesql @DynamicSQL;
    """
    # 执行SQL查询
    try:
        cursor.execute(sql)
        # 获取查询结果
        result = cursor.fetchall()
        # 将列表转换为DataFrame
        df = pd.DataFrame.from_records(result, columns=['表名', '字段名','字段类型', '非空占比', '唯一值数量','唯一值集合'])
        df1 = df
        # 计算非空占比不是100%的字段
        non_100_percent_fields = df1[df1['非空占比'] != '100.00%']

        # 统计这些字段的数量
        num_non_100_percent_fields = len(non_100_percent_fields)
        # 整理为一句话并显示结果
        fields_list = ', '.join(non_100_percent_fields['字段名'].tolist())
        print(f"共有 {num_non_100_percent_fields} 个字段的非空占比不是100%，这些字段分别是：{fields_list}")
        # 增加一列，用于标记是否为高基数字段
        df1['字段分类'] = df1['唯一值数量'].apply(lambda x: '多值变量' if x > 250 else '非多值变量')
        # 替换大于50类为略
        df1['唯一值集合'] = df1['唯一值集合'].apply(lambda x: '略' if x == '大于250种类' else x)
        # 按照非空占比进行排序
        df1 = df1.sort_values(by=['字段分类','非空占比','唯一值数量'], ascending=[True,False,False])
        # 将非空占比转换为百分数并保留两位小数
        df1['非空占比'] = df['非空占比'].apply(lambda x: '{:.2f}%'.format(float(x)))
        # 更换列顺序
        df1 = df1[['表名','字段分类','字段名','字段类型','非空占比','唯一值数量','唯一值集合']]
        # 将结果写入Excel并保存到本地
        df1.to_excel(f'./Member_data/{table_name}.xlsx', index=False)
    except pyodbc.ProgrammingError as e:
        print(f"表 {table_name} 查询出错，错误信息为：{e}")
        failed_tables.append(table_name)
        continue
    except :
        print(f"表 {table_name} 查询超时，已跳过")
        timeout_tables.append(table_name)
        continue
# 关闭游标和连接
cursor.close()
conn.close()

In [5]:
# 将member_data文件夹中的所有Excel文件合并为一个Excel文件
# 获取member_data文件夹中的所有Excel文件
import glob
file_list = glob.glob('./Member_data/*.xlsx')
# 读取所有Excel文件
df_list = [pd.read_excel(file) for file in file_list]
# 合并所有Excel文件
df = pd.concat(df_list)
# 将结果写入Excel并保存到本地
df.to_excel('./Member_data/Member_data.xlsx', index=False)